In [5]:
import pandas as pd
import numpy as np
import json
import re
import time
import pickle
import os

# Filtering for Chinese Businesses

In [3]:
def get_busdata():
    try:
        with open('/mnt/data/public/yelp/challenge12/yelp_dataset/'
                      'yelp_academic_dataset_business.json', 'r') as f:
                business = f.readlines()

        businesses = []
        for i in range(len(business)):
            data = json.loads(business[i])
            businesses.append(data)

        bus_df = pd.DataFrame(businesses)
        chinese = bus_df.dropna()
        chinese = chinese[chinese['categories'].str.contains('Chinese|chinese')]
        bus = chinese

    except:
        bus = pd.read_csv('chinese_bus.csv')
    
    return bus

In [14]:
if os.path.isfile('chinese_bus.csv'):
    bus = pd.read_csv('chinese_bus.csv')\
            .rename(columns = {'Unnamed: 0': 'review_id'})
    
else:
    bus = get_revdata()

In [15]:
bus.head()

,review_id,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,73,f2FfutZhb4F-m1Ob0EdYaw,China Palace,Shadyside,5440 Walnut St,Pittsburgh,PA,15232,40.450866,-79.933919,3.0,110,1,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...","Asian Fusion, Caterers, Fast Food, Chinese, Re...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
1,109,SUhBnQdG8MWkmjM6wsnVdw,Tang Dynasty Restaurant And Bar,NaN,2515 N Scottsdale Rd,Scottsdale,AZ,85257,33.475349,-111.925657,3.5,28,1,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...","Chinese, Bars, Karaoke, Nightlife, Restaurants","{'Monday': '12:0-22:30', 'Tuesday': '11:0-23:0..."
2,214,5ZqybrqiTdbANn2yI3B3VA,Great Wall,Allied Dunn's Marsh,4515 Verona Rd,Madison,WI,53711,43.035241,-89.453599,3.5,22,1,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...","Restaurants, Chinese","{'Monday': '10:30-21:30', 'Tuesday': '10:30-21..."
3,331,J8PbfZbU7XTs-h55ezuQcQ,CHANG HUA GARDEN,NaN,811 S Gammon Rd,Fitchburg,WI,53719,43.047406,-89.502630,4.5,3,0,{'RestaurantsAttire': 'casual'},"Chinese, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
4,372,YkAIlxYZ1guSqbbowU9X4g,Luckee,Entertainment District,328 Wellington Street W,Toronto,ON,M5V 3T4,43.644697,-79.392395,3.5,171,1,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...","Restaurants, Chinese, Dim Sum, Breakfast & Brunch","{'Monday': '16:0-1:0', 'Tuesday': '12:0-1:0', ..."


# Review Data

In [11]:
bus_id_list = bus.business_id.tolist()

def get_revdata():
    try:
        with open('/mnt/data/public/yelp/challenge12/yelp_dataset/'
                  'yelp_academic_dataset_review.json') as f:
            review = f.readlines()

        json.loads(review[0])['business_id'] in bus_id_list

        # Filtering for reviews for businesses found in Chinese business list
        rev = {}
        for i in review:
            x = json.loads(i)
            if x['business_id'] in bus_id_list:
                for d in x:
                    rev[x['review_id']] = x

        reviews = pd.DataFrame(rev).T.drop(columns = 'review_id')
        rev = reviews

    except:
        rev = pd.read_csv('reviews.csv')\
        .rename(columns = {'Unnamed: 0': 'review_id'})
        
    return rev

In [12]:
if os.path.isfile('reviews.csv'):
    rev = pd.read_csv('reviews.csv')\
            .rename(columns = {'Unnamed: 0': 'review_id'})
    
else:
    rev = get_revdata()

In [13]:
rev.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,oz66Z8p9Etq0WbcZVCmm7w,0pf5VuzE4_1pwj5NJHG5TQ,OX0T9dWI8b7meu-ljTo22A,5,2016-05-01,"Friendliest staff, no matter how swamped they ...",0,0,0
1,8k2WpYJFfsPL3kupmvGjWA,0pf5VuzE4_1pwj5NJHG5TQ,szhJLmdLDVFTevm8fu0T4A,5,2016-05-01,Best place for dim sum in the valley. Prefer ...,0,0,0
2,JvCsQXCkTkybpkJd8oSimw,0pf5VuzE4_1pwj5NJHG5TQ,A2pHOVp0zh7grb1bbZxoBQ,4,2016-06-13,Love the house chicken and chicken fried rice....,0,0,0
3,mqhCY5oLA1LXP6gZ3JQDTQ,Yy_iGXxLpL6tYDQoE-6XVg,JPfi__QJAaRzmfh5aOyFEw,3,2017-01-31,The spicy wonton was my favorite. We ordered a...,0,0,0
4,sjvlLg6iKlsw4pje1-FElg,OQT9DjfBrzrwOEdVJjuYIA,HDv-KchmLSlRr5MXxRkNqQ,3,2012-08-18,I have passed this spot hundreds of times but ...,3,0,0


# Data Merge

In [17]:
df = rev.merge(bus, on = "business_id")

In [18]:
df = df.loc[(df['city'] == 'Las Vegas') & (df['neighborhood'] == 'Chinatown')]

In [20]:
try:
    df = df.drop(columns = 'Unnamed: 0')

except:
    pass

In [21]:
if os.path.isfile('vegas_chinatown.csv'):
    pass

else:
    df.to_csv('vegas_chinatown.csv')